In [526]:
import sqlite3 as db
import pandas as pd
import featuretools as ft
import json

<center><h2><b>Leer DB</b></h2></center>

In [527]:
# Hacer una query SQL
def sql_query(q):
    conn = db.connect('../db/sqlite/eicu_v2_0_1.sqlite3')
    df = pd.read_sql_query(q, conn)
    conn.close()
    
    return df

# Leer todos los CSV
def read_csvs():
    #import os
    #print( os.getcwd())
    datasets = [ 'admissiondrug', 'admissionDx', 'allergy', 'apacheApsVar', 'apachePatientResult', 'apachePredVar', 'carePlanCareProvider', 'carePlanEOL', 'carePlanGeneral',
                 'carePlanGoal', 'carePlanInfectiousDisease', 'customLab', 'diagnosis', 'hospital', 'infusiondrug', 'intakeOutput', 'lab', 'medication', 'microLab', 'note',
                 'nurseAssessment', 'nurseCare', 'nurseCharting', 'pastHistory', 'patient', 'physicalExam', 'respiratoryCare', 'respiratoryCharting', 'treatment', 'vitalAperiodic',
                 'vitalPeriodic']

    dfs = {}

    for ds_name in datasets:
        dfs[ds_name] = pd.read_csv('../db/csv/' + ds_name + '.csv')
    
    return dfs

dfs = read_csvs()
has_dropped_keys = False # Para que no se droppeen cada vez que se ejecuta la celda siguiente

/tmp/ipykernel_5156/2990043772.py:21: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  dfs[ds_name] = pd.read_csv('../db/csv/' + ds_name + '.csv')


In [528]:
# Dropear IDs de todas las tablas (primera columna)
def drop_keys(dfs):
    for df_key in dfs.keys():
        if df_key not in ('hospital', 'patient'): # No dropear hospitalId o patientUnitStayId
            df = dfs[df_key]
            dfs[df_key] = df.drop(columns=[df.columns.values[0]])

if not has_dropped_keys:
    drop_keys(dfs)
    has_dropped_keys = True

<center><h2><b>admissionDx</b></h2></center>

In [529]:
# TODO: ? Posiblemente dropear en virtud de patient.apacheAdmissionDx

dfs['admissionDx']

,patientunitstayid,admitdxenteredoffset,admitdxpath,admitdxname,admitdxtext
0,2900423,162,admission diagnosis|Non-operative Organ Systems|Organ System|Cardiovascular,Cardiovascular,Cardiovascular
1,2900423,162,admission diagnosis|Was the patient admitted from the O.R. or went to the O.R. within 4 hours of admission?|No,No,No
2,2900423,162,"admission diagnosis|All Diagnosis|Non-operative|Diagnosis|Cardiovascular|Sepsis, pulmonary","Sepsis, pulmonary","Sepsis, pulmonary"
3,2902156,944,"admission diagnosis|All Diagnosis|Non-operative|Diagnosis|Cardiovascular|Rhythm disturbance (atrial, supraventricular)","Rhythm disturbance (atrial, supraventricular)","Rhythm disturbance (atrial, supraventricular)"
4,2902156,944,admission diagnosis|Non-operative Organ Systems|Organ System|Cardiovascular,Cardiovascular,Cardiovascular
...,...,...,...,...,...
7573,2898513,50,"admission diagnosis|All Diagnosis|Non-operative|Diagnosis|Gastrointestinal|Encephalopathy, hepatic","Encephalopathy, hepatic","Encephalopathy, hepatic"
7574,2899929,11,admission diagnosis|Was the patient admitted from the O.R. or went to the O.R. within 4 hours of admission?|Yes,Yes,Yes
7575,2899929,11,admission diagnosis|Elective|Yes,Yes,Yes
7576,2899929,11,"admission diagnosis|All Diagnosis|Operative|Diagnosis|Genitourinary|TURP, transurethral prostate resection for benign prostatic hypertrophy","TURP, transurethral prostate resection for benign prostatic hypertrophy","TURP, transurethral prostate resection for benign prostatic hypertrophy"


In [530]:
# Drop columns
dfs['admissionDx'] = dfs['admissionDx'].drop(columns=[
    'admitdxtext', # En la gran mayoría de los casos es igual a admitdxname
], axis=1)

In [531]:
#
# Nota: Es una mierda porque tiene mucho más sentido con onehot, pero hay un huevo de diagnósticos y como 10~20 notas adicionales
# TODO: diag1 = 4 ??
#

# admitdxpath - admitdxname: Cada patientunistayid tiene n filas relacionadas con todos las diagnosis encontradas.
def parse_admitdx(group):
    pusID, group_df = group
    diagnosis = []
    additionalinfo = []
    
    for i, row in group_df.iterrows():
        if row['admitdxname'] in ('No', 'Yes'):
            path_parts = row['admitdxpath'].split('|')
            mode = path_parts[-2]
            
            additionalinfo.append(mode + ' ' + row['admitdxname'])
        else:
            diagnosis.append(row['admitdxname'])
            
    # Rellenar 3 huecos en diagnosis y 2 en additionalinfo
    diagnosis = (sorted(diagnosis) + ['No' for _ in range(3)])[:3]
    additionalinfo = (sorted(additionalinfo) + ['No' for _ in range(3)])[:3]
    
    return [pusID] + diagnosis + additionalinfo

new_df = pd.DataFrame(columns=['patientunitstayid', 'diag1', 'diag2', 'diag3', 'additional1', 'additional2', 'additional3'])
for i,  group in enumerate(dfs['admissionDx'].groupby('patientunitstayid')):
    row = parse_admitdx(group)
    new_df.loc[i] = row

new_df

,patientunitstayid,diag1,diag2,diag3,additional1,additional2,additional3
0,141765,Cardiovascular,"Rhythm disturbance (atrial, supraventricular)",No,Was the patient admitted from the O.R. or went to the O.R. within 4 hours of admission? No,No,No
1,143870,Cardiovascular,"Endarterectomy, carotid",No,Elective No,Was the patient admitted from the O.R. or went to the O.R. within 4 hours of admission? Yes,No
2,144815,Neurologic,"Overdose, other toxin, poison or drug",No,Was the patient admitted from the O.R. or went to the O.R. within 4 hours of admission? No,No,No
3,145427,"GI perforation/rupture, surgery for",Gastrointestinal,No,Elective No,Was the patient admitted from the O.R. or went to the O.R. within 4 hours of admission? Yes,No
4,147307,Cardiovascular,"Endarterectomy, carotid",No,Elective Yes,Was the patient admitted from the O.R. or went to the O.R. within 4 hours of admission? Yes,No
...,...,...,...,...,...,...,...
2216,3351763,"Fistula/abscess, surgery for (not inflammatory bowel disease)",Gastrointestinal,No,Elective Yes,Was the patient admitted from the O.R. or went to the O.R. within 4 hours of admission? Yes,No
2217,3352230,4,"CABG alone, coronary artery bypass grafting",Cardiovascular,Elective Yes,Internal mammary artery graft? Yes,Pre-op MI during current hospitalization Yes
2218,3352231,Cardiovascular,"Infarction, acute myocardial (MI)",inferior,Thrombolytic Therapy received within 24 hours No,Was the patient admitted from the O.R. or went to the O.R. within 4 hours of admission? No,No
2219,3352333,GI obstruction,Gastrointestinal,No,Was the patient admitted from the O.R. or went to the O.R. within 4 hours of admission? No,No,No


In [532]:
dfs['admissionDx']

,patientunitstayid,admitdxenteredoffset,admitdxpath,admitdxname
0,2900423,162,admission diagnosis|Non-operative Organ Systems|Organ System|Cardiovascular,Cardiovascular
1,2900423,162,admission diagnosis|Was the patient admitted from the O.R. or went to the O.R. within 4 hours of admission?|No,No
2,2900423,162,"admission diagnosis|All Diagnosis|Non-operative|Diagnosis|Cardiovascular|Sepsis, pulmonary","Sepsis, pulmonary"
3,2902156,944,"admission diagnosis|All Diagnosis|Non-operative|Diagnosis|Cardiovascular|Rhythm disturbance (atrial, supraventricular)","Rhythm disturbance (atrial, supraventricular)"
4,2902156,944,admission diagnosis|Non-operative Organ Systems|Organ System|Cardiovascular,Cardiovascular
...,...,...,...,...
7573,2898513,50,"admission diagnosis|All Diagnosis|Non-operative|Diagnosis|Gastrointestinal|Encephalopathy, hepatic","Encephalopathy, hepatic"
7574,2899929,11,admission diagnosis|Was the patient admitted from the O.R. or went to the O.R. within 4 hours of admission?|Yes,Yes
7575,2899929,11,admission diagnosis|Elective|Yes,Yes
7576,2899929,11,"admission diagnosis|All Diagnosis|Operative|Diagnosis|Genitourinary|TURP, transurethral prostate resection for benign prostatic hypertrophy","TURP, transurethral prostate resection for benign prostatic hypertrophy"


<center><h2><b>allergy</b></h2></center>

In [533]:
dfs['allergy']

,patientunitstayid,allergyoffset,allergyenteredoffset,allergynotetype,specialtytype,usertype,rxincluded,writtenineicu,drugname,allergytype,allergyname,drughiclseqno
0,243097,2549,2552,Comprehensive Progress,eCM Primary,THC Nurse,True,True,NaN,Non Drug,penicillins,NaN
1,243097,1288,1294,Comprehensive Progress,eCM Primary,THC Nurse,True,True,CODEINE PHOSPHATE,Drug,CODEINE PHOSPHATE,1721.0
2,243097,2549,2552,Comprehensive Progress,eCM Primary,THC Nurse,True,True,CODEINE PHOSPHATE,Drug,CODEINE PHOSPHATE,1721.0
3,243097,21,28,Admission,eCM Primary,THC Nurse,True,True,NaN,Non Drug,penicillins,NaN
4,243097,3988,3989,Comprehensive Progress,eCM Primary,THC Nurse,True,True,CODEINE PHOSPHATE,Drug,CODEINE PHOSPHATE,1721.0
...,...,...,...,...,...,...,...,...,...,...,...,...
2470,3351763,14,51,Admission,eCM Primary,THC Nurse,False,True,NaN,Non Drug,latex,NaN
2471,3351763,14,51,Admission,eCM Primary,THC Nurse,False,True,LEVAQUIN,Drug,LEVAQUIN,12383.0
2472,3353113,77,86,Admission,eCM Primary,THC Nurse,False,True,NaN,Non Drug,Contrast Dye,NaN
2473,3353113,77,86,Admission,eCM Primary,THC Nurse,False,True,PENICILLIN G BENZATHINE,Drug,PENICILLIN G BENZATHINE,3941.0


In [534]:
dfs['allergy'] = dfs['allergy'].drop(columns=[
    'allergyoffset', # Preguntar
    'specialtytype', # Poca varianza 2453 – 22
    'drughiclseqno'
], axis=1)

In [535]:
pd.set_option('display.max_rows', 400)
pd.set_option('display.max_colwidth', None)
#sql_query('SELECT * FROM allergy GROUP BY drugname, allergytype').head(400)

dfs['allergy']['drugname'].isna().value_counts()

False    1480
True      995
Name: drugname, dtype: int64

<center><h2><b>apacheapsvar</b></h2></center>

<center><h2><b>apachepatientresult</b></h2></center>

<center><h2><b>Apachepredvar</b></h2></center>

<center><h2><b>careplancareprovider</b></h2></center>

<center><h2><b>Careplaneol</b></h2></center>

<center><h2><b>careplangeneral</b></h2></center>

<center><h2><b>careplangoal</b></h2></center>

<center><h2><b>careplaninfectiousdisease</b></h2></center>

<center><h2><b>customlab</b></h2></center>

<center><h2><b>diagnosis</b></h2></center>

<center><h2><b>hospital</b></h2></center>

<center><h2><b>Infusiondrug</b></h2></center>

<center><h2><b>intakeoutput</b></h2></center>

In [536]:
dfs['intakeOutput'] = dfs['intakeOutput'].drop(columns=[
    'intaketotal', # Computada en otra variable
    'outputtotal', # Computada en otra variable
    'cellpath', # Redundante en celltext
    'cellvaluetext' # dependiente de cellvaluenumeric
], axis=1)

In [537]:
dfs['intakeOutput']

,patientunitstayid,intakeoutputoffset,dialysistotal,nettotal,intakeoutputentryoffset,celllabel,cellvaluenumeric
0,147307,-394,0.0,0.00,-394,Bodyweight (lb),159.80
1,147307,-394,0.0,0.00,-394,Bodyweight (kg),72.50
2,211715,1533,0.0,120.00,1533,P.O.,120.00
3,219981,6504,0.0,120.00,6504,P.O.,120.00
4,158057,624,0.0,0.00,624,Bodyweight (lb),359.00
...,...,...,...,...,...,...,...
100461,3348409,1930,0.0,68.75,1930,Crystalloids,68.75
100462,3329305,1366,0.0,0.00,1366,Crystalloids,150.00
100463,3346866,2980,0.0,130.00,2980,P.O.,480.00
100464,3346866,2980,0.0,130.00,2980,Urine,350.00


<center><h2><b>lab</b></h2></center>

In [538]:
# TODO: Mirar relación entre labmeasurenamesystem y labmeasurenameinterface

dfs['lab'] = dfs['lab'].drop(columns=[
    'labtypeid', # Irrelevante
    'labname', # Irrelevante
    'labresulttext', # Redundante con labResult
    'labresultrevisedoffset' # Reundante con labresultoffset
], axis=1)

In [539]:
dfs['lab'] 

,patientunitstayid,labresultoffset,labresult,labmeasurenamesystem,labmeasurenameinterface
0,1754323,-647,38.30,%,%
1,1754323,-647,181.00,K/mcL,k/mm cu
2,1754323,-647,4.86,M/mcL,m/mm cu
3,1754323,-647,8.70,%,%
4,1754323,-647,30.40,g/dL,%
...,...,...,...,...,...
434655,2754778,2041,91.00,mg/dL,mg/dL
434656,2754778,37,5.92,ng/mL,ng/mL
434657,2754778,37,92.00,fL,fL
434658,2754778,236,119.00,mg/dL,mg/dL


<center><h2><b>medication</b></h2></center>

In [540]:
dfs['medication']

,patientunitstayid,drugorderoffset,drugstartoffset,drugivadmixture,drugordercancelled,drugname,drughiclseqno,dosage,routeadmin,frequency,loadingdose,prn,drugstopoffset,gtc
0,141765,134,1396,No,No,WARFARIN SODIUM 5 MG PO TABS,2812.0,5 3,PO,NaN,NaN,No,2739,0
1,141765,1,-188,No,No,5 ML VIAL : DILTIAZEM HCL 25 MG/5ML IV SOLN,182.0,15 3,IV,Once PRN,NaN,Yes,171,38
2,141765,115,856,No,No,ASPIRIN EC 81 MG PO TBEC,1820.0,81 3,PO,Daily,NaN,No,2739,0
3,141765,114,316,No,No,DILTIAZEM HCL 30 MG PO TABS,182.0,30 3,PO,Q6H SCH,NaN,No,2739,0
4,141765,115,856,No,No,LISINOPRIL 5 MG PO TABS,132.0,5 3,PO,Daily,NaN,No,2428,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75599,3353113,3367,3789,No,Yes,NaN,6249.0,1 EACH,TRANSDERM,DAILY,NaN,No,0,97
75600,3353113,2,-55,No,No,NaN,20971.0,100 ML,INTRAVEN,DRIP,NaN,No,2721,83
75601,3353113,2,-2258,No,Yes,LORAZEPAM,4846.0,2-4 MG,INTRAVEN,NaN,NaN,Yes,0,83
75602,3353113,2,-86,No,Yes,NaN,23379.0,20 MG,INTRAMUSC,Q4HRSPRN,NaN,Yes,-65,80


In [541]:
# TODO: Parsear dosage con drugname

dfs['medication'] = dfs['medication'].drop(columns=[
    'drugorderoffset', # Irrelevante
    'drugivadmixture', # No parece relevante
    'drugordercancelled', # Irrelevante
    'drugname', # Muy incompleta y redundante con drughiclseqno
    'loadingdose', # Mucho NULL
    'prn', # ? Redundante con frequency
    'gtc' # ? No parece que tenga relevancia
], axis=1)

<center><h2><b>microLab</b></h2></center>

In [542]:
# TODO ?

<center><h2><b>note</b></h2></center>

In [543]:
dfs['note'] = dfs['note'].drop(columns=[
    'noteenteredoffset', # Irrelevante
    'notetype', # No parece relevante
    'notevalue', # Parece tener información sobre notetext
], axis=1)

<center><h2><b>nurseAssessment</b></h2></center>

In [544]:
# TODO: ? Parece que hay múltiples assessments de diferentes enfermeras al mismo tiempo con diagnósticos diferentes.
# TODO: Unir celllabel	cellattribute	cellattributevalue
# TODO: Hacer algo con cellattributepath

dfs['nurseAssessment'] = dfs['nurseAssessment'].drop(columns=[
    'nurseassessentryoffset', # Irrelevante
], axis=1)

<center><h2><b>nurseCare</b></h2></center>

In [545]:
dfs['nurseCare'] = dfs['nurseCare'].drop(columns=[
    'nursecareentryoffset', # Irrelevante
    'celllabel' # cellattribute es más detallada
], axis=1)

<center><h2><b>nurseCharting</b></h2></center>

In [546]:
# Hacer algo con esta mierda

dfs['nurseCharting'] = dfs['nurseCharting'].drop(columns=[
    'nursingchartentryoffset' # Irrelevante
], axis=1)

<center><h2><b>pastHistory</b></h2></center>

In [547]:
dfs['pastHistory'] = dfs['pastHistory'].drop(columns=[
    'pasthistoryenteredoffset', # Irrelevante
    'pasthistoryvaluetext' # Igual que pasthistoryvalue
], axis=1)

<center><h2><b>patient</b></h2></center>

In [548]:
# TODO: Feature idea - BMI
# TODO: ? Cual es más importante, unitDischargeOffset u hospitalDischargeOffset
import math

def clean_age(row):
    age = row['age']
    
    if age == '':
        return 50 # TODO: Median
    elif age == '> 89':
        return 100
    elif math.isnan(float(age)):
        return 50 # ""
    else:
        return int(age)

# Drop
dfs['patient'] = dfs['patient'].drop(columns=[
    'wardid', # ? Tiene pinta de que no va a ser muy importante
    'patienthealthsystemstayid', # Irrelevante
    'hospitaldischargeyear', # Irrelevante. Solo hay datos de los años 2014 y 2015
    'hospitaladmittime24', # Drop a priori. No parece importante
    'hospitaldischargetime24', # Drop a priori. No parece importante
    'hospitaldischargelocation', # No parece muy relevante
    'unittype', # Parece que ya está presente en otras partes
    'unitadmittime24', # No parece muy relevante
    'unitadmitsource', # ?
    'unitstaytype', # No parece muy relevante
    'dischargeweight', # No parece que sea muy relevante el peso cuando esté muerto
    'unitdischargetime24', # Irrelevante
    'unitdischargelocation', # Irrelevante
    'hospitaldischargestatus', # Irrelevante
    'unitdischargestatus',
    'uniquepid'
], axis=1)

# Eliminar NaN TODO
dfs['patient'] = dfs['patient'].dropna()

# Age
dfs['patient']['age'] = dfs['patient'].apply(clean_age, axis=1)

<center><h2><b>physicalExam</b></h2></center>

In [549]:
# TODO: Darle una limpieza fuerte

dfs['physicalExam'] = dfs['physicalExam'].drop(columns=[
    'physicalexamoffset', # Irrelevante
], axis=1)

<center><h2><b>respiratoryCare</b></h2></center>

In [550]:
dfs['respiratoryCare'] = dfs['respiratoryCare'].drop(columns=[
    'airwaysize', 'airwayposition', 'cuffpressure', 'apneaparams', 'lowexhmvlimit', 'hiexhmvlimit', 'lowexhtvlimit', 'hipeakpreslimit', 'lowpeakpreslimit',
    'hirespratelimit', 'lowrespratelimit', 'sighpreslimit', 'lowironoxlimit', 'highironoxlimit', 'meanairwaypreslimit', 'peeplimit', 'cpaplimit',
    'setapneainterval', 'setapneatv', 'setapneaippeephigh', 'setapnearr', 'setapneapeakflow', 'setapneainsptime', 'setapneaie', 'setapneafio2'
], axis=1)

<center><h2><b>respiratoryCharting</b></h2></center>

In [551]:
# TODO: ?

dfs['respiratoryCharting'] = dfs['respiratoryCharting'].drop(columns=[
    'respchartentryoffset' # Irrelevante
], axis=1)

<center><h2><b>treatment</b></h2></center>

In [552]:
# TODO: ?

dfs['treatment'] = dfs['treatment'].drop(columns=[
    'activeupondischarge', # Irrelevante    
], axis=1)

<center><h2><b>vitalAperiodic</b></h2></center>

In [553]:
# TODO

dfs['vitalAperiodic'] = dfs['vitalAperiodic'].drop(columns=[
], axis=1)

<center><h2><b>vitalPeriodic</b></h2></center>

In [554]:
# TODO

dfs['vitalPeriodic'] = dfs['vitalPeriodic'].drop(columns=[
], axis=1)

---

<center><h2><b>Exportar a CSV</b></h2></center>

In [555]:
for df_name in dfs:
    dfs[df_name].to_csv('../db/csv_clean/' + df_name + '.csv', index=False)